# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

In [40]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

from adamacs.pipeline import subject, session, surgery, scan, equipment
from adamacs import utility
from adamacs.ingest import session as isess
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath())

In [ ]:
# subject.Subject.drop()
# subject.Mutation.drop()
# subject.Protocol.drop()
# subject.Line.drop()
# subject.SubjectGenotype.drop()
# equipment.Equipment.drop()
# scan.Scan.drop()
# session.Session.drop()
imaging.Processing.drop()

In [ ]:
scan.Scan.describe()

In [ ]:
dj.Diagram(subject) + dj.Diagram(protocol)

Manual entry:

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass()  # Put your password in the prompt
dj.conn()

from adamacs.pipeline import subject, session, surgery, scan, equipment
from adamacs import utility
from adamacs.ingest import session as isess
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath())


## Ingesting scan

Ingest all scans associated with a given session ID.

In [ ]:
# isess.ingest_session_scan('sess9FB2LN5C', verbose=True)
isess.ingest_session_scan('sess9FF6TL96', verbose=True) #TR: needs to be SessioID ("sess...")

In [ ]:
# isess.ingest_session_scan('sess9FB2LN5C', verbose=True)
isess.ingest_session_scan('sess9FF6U114', verbose=True) #TR: needs to be SessioID ("sess...")

In [42]:
session.Session * session.SessionDirectory

session_id,subject PyRat import uses this for earmark value,session_datetime,session_dir Path to the data directory for a session,user_id
sess9FF6TL96,ROS-1438,2022-11-22 00:00:00,/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6TL96_sess9FF6TL96,1
sess9FF6U114,ROS-1438,2022-11-22 00:00:00,/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6U114_sess9FF6U114,6


In [ ]:
session.SessionDirectory()

In [ ]:
key='scan9FF6TL96'
(scan.Scan & f'scan_id=\"{key}\"')

Simple user look-up to get user_id from shorthand

In [ ]:
key='JJ'
query = (subject.User & f'initials=\"{key}\"')
id = query.fetch("user_id")[0]
print(key + ' = ' + str(id))

In [ ]:
key='Tobias*'
query = (subject.User & f'name LIKE \"{key}\"')
id = query.fetch("user_id")[0]
print(key + ' = ' + str(id))

In [ ]:
key='%Ji%'
query = (subject.User & f'name LIKE \"{key}\"')
query.fetch("email")

In [ ]:
query

In [ ]:
query = subject.Subject() & '"ROS-1380"'
query.fetch()

In [ ]:
subject.Subject() & "subject = 'ROS-1346'"

##### Some placeholders for equipment and location during development

In [43]:
scan.ScanInfo()

session_id,scan_id,nfields number of fields,nchannels number of channels,ndepths Number of scanning depths (planes),nframes number of recorded frames,nrois number of ROIs (see scanimage's multi ROI imaging),x (um) ScanImage's 0 point in the motor coordinate system,y (um) ScanImage's 0 point in the motor coordinate system,z (um) ScanImage's 0 point in the motor coordinate system,fps (Hz) frames per second - Volumetric Scan Rate,bidirectional true = bidirectional scanning,usecs_per_line microseconds per scan line,fill_fraction raster scan temporal fill fraction (see scanimage),scan_datetime datetime of the scan,scan_duration (seconds) duration of the scan
sess9FF6TL96,scan9FF6TL96,1,2,1,9250,0,nan,nan,nan,15.3845,1,250.001,0.95,None,601.255


In [ ]:
scan.Scan()

In [ ]:
scan.Scan * scan.ScanInfo

Note the relative path below:

In [ ]:
#temporary step - insert placeholder values
equipment_placeholder = "Equipment"
location_placeholder = "Location"
from adamacs.pipeline import Equipment, Location
Equipment.insert1({'scanner' : equipment_placeholder}, skip_duplicates=True)
Location.insert1({'anatomical_location': location_placeholder}, skip_duplicates=True) 

In [ ]:
Equipment()
